#Interactive Visual Analytics with Folium

The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using matplotlib and seaborn and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using Folium.

## Objectives

This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.

In [1]:
# Install folium and pandas in Colab
!pip install folium pandas

import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition
from folium.features import DivIcon

# Download and read the CSV file directly using pandas
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
spacex_df = pd.read_csv(url)

# Check dataframe head
spacex_df.head()


,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


Now, you can take a look at what are the coordinates for each site.

In [2]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.







We first need to create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [3]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use folium.Circle to add a highlighted circle area with a text label on a specific coordinate. For example,

In [4]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.

Now, let's add a circle for each launch site in data frame launch_sites

TODO: Create and add folium.Circle and folium.Marker for each launch site on the site map

An example of folium.Circle:

folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))

An example of folium.Marker:

folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))

In [11]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site in the dataframe
for _, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']

    folium.Circle(
        location=[lat, lon],
        radius=5000,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.15,  # very transparent fill
        opacity=0.15,       # very transparent outline
        popup=site_name
    ).add_to(site_map)

# Display the map
site_map

Now, you can explore the map by zoom-in/out the marked areas , and try to answer the following questions:

Are all launch sites in proximity to the Equator line?
Are all launch sites in very close proximity to the coast?
Also please try to explain your findings.

In [12]:
# Initialize the map (center it around NASA coordinate)
nasa_coordinate = [28.5721, -80.648]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each row in dataframe
for _, row in spacex_df.iterrows():
    lat = row['Lat']
    lon = row['Long']
    site_name = row['Launch Site']
    launch_status = row['class']  # 1 = success, 0 = fail

    # Choose marker color based on success or failure
    if launch_status == 1:
        marker_color = 'green'
        popup_text = f"{site_name} - Success"
    else:
        marker_color = 'red'
        popup_text = f"{site_name} - Failure"

    # Add a marker with color and popup
    folium.Marker(
        location=[lat, lon],
        popup=popup_text,
        icon=folium.Icon(color=marker_color, icon='rocket', prefix='fa')
    ).add_to(site_map)

site_map

Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates. Recall that data frame spacex_df has detailed launch records, and the class column indicates if this launch was successful or not

In [13]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. If a launch was successful (class=1), then we use a green marker and if a launch was failed, we use a red marker (class=0)

Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.







Let's first create a MarkerCluster object

In [14]:
marker_cluster = MarkerCluster()

Create a new column in spacex_df dataframe called marker_color to store the marker colors based on the class value

In [15]:
# Apply a function to assign marker colors based on 'class' column
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


For each launch result in spacex_df data frame, add a folium.Marker to marker_cluster

In [17]:
# Initialize the map centered around NASA coordinate
nasa_coordinate = [28.5721, -80.648]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Initialize MarkerCluster
marker_cluster = MarkerCluster().add_to(site_map)

# Loop through each launch record and add a marker to the cluster
for _, row in spacex_df.iterrows():
    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} - {'Success' if row['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color=row['marker_color'], icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Display the map
site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.

In [18]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance (in km) between two points on the earth.
    """
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r


In [19]:
# Extract unique launch sites with their coordinates
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates().reset_index(drop=True)


In [21]:


# Create an empty list to store distances
distances = []

# Loop over each site
for i, site1 in launch_sites.iterrows():
    for j, site2 in launch_sites.iterrows():
        if site1['Launch Site'] != site2['Launch Site']:
            dist = haversine(site1['Long'], site1['Lat'], site2['Long'], site2['Lat'])
            distances.append({
                'Site 1': site1['Launch Site'],
                'Site 2': site2['Launch Site'],
                'Distance (km)': dist
            })

# Create DataFrame of distances
distances_df = pd.DataFrame(distances)

# Display distances
distances_df.head()


,Site 1,Site 2,Distance (km)
0,CCAFS LC-40,VAFB SLC-4E,3825.840279
1,CCAFS LC-40,KSC LC-39A,6.899304
2,CCAFS LC-40,CCAFS SLC-40,0.112488
3,VAFB SLC-4E,CCAFS LC-40,3825.840279
4,VAFB SLC-4E,KSC LC-39A,3819.052794


Next, we need to explore and analyze the proximities of launch sites.

Let's first add a MousePosition on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)

In [23]:
from folium.plugins import MousePosition

# Formatter for 5 decimal places
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

mouse_position = MousePosition(
    position='topright',
    separator=', ',      # separates latitude and longitude nicely
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map



Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.

In [24]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

In [25]:
from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance (in km) between two points on the earth.
    """
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371  # Earth radius in km
    return c * r


In [26]:
launch_site_lat = 28.5623    # example launch site latitude
launch_site_lon = -80.5774   # example launch site longitude

coastline_lat = 28.56367     # closest coastline latitude (example)
coastline_lon = -80.57163    # closest coastline longitude (example)


In [27]:
distance_coastline = haversine(launch_site_lon, launch_site_lat, coastline_lon, coastline_lat)
print(f"Distance from launch site to coastline: {distance_coastline:.2f} km")


Distance from launch site to coastline: 0.58 km


In [28]:
import folium
from folium.features import DivIcon

# Add the launch site marker (optional)
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup='Launch Site',
    icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
).add_to(site_map)

# Add marker on the coastline
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html=f'<div style="font-size: 12pt; color:#d35400;"><b>{distance_coastline:.2f} KM</b></div>',
    )
)
distance_marker.add_to(site_map)

# Optional: Add a marker on coastline itself for clarity
folium.CircleMarker(
    [coastline_lat, coastline_lon],
    radius=6,
    color='red',
    fill=True,
    fill_color='red',
    fill_opacity=0.7,
    popup='Closest Coastline'
).add_to(site_map)

# Show map
site_map


 Draw a PolyLine between a launch site to the selected coastline point

In [29]:
# Define the coordinates list (launch site to coastline)
coordinates = [
    [launch_site_lat, launch_site_lon],
    [coastline_lat, coastline_lon]
]

# Create the PolyLine object
lines = folium.PolyLine(locations=coordinates, weight=3, color='blue', opacity=0.7)

# Add the PolyLine to the map
site_map.add_child(lines)

# Display the map
site_map
